When you submit an experiment, you use its run context to initialize and end the experiment run that is tracked in Azure Machine Learning, as shown in the following code sample:

In [2]:
from azureml.core import Experiment, Workspace

In [3]:
ws = Workspace.from_config()
ws

Workspace.create(name='aml-workspace', subscription_id='3ed3266f-ff5e-4b56-b844-7568f3957f98', resource_group='am-rg')

In [4]:
# create an experiment variable
experiment = Experiment(
    workspace=ws,
    name="my-experiment"
)

# start the experiment
run = experiment.start_logging()
# experiment goes here

# end experiment
run.complete()


## Logging Metrics and Creating Outputs

### logging Metrics

Every experiment generates log files that include the messages that would be written to the terminal during interactive execution. This enables you to use simple print statements to write messages to the log. However, if you want to record named metrics for comparison across runs, you can do so by using the Run object; which provides a range of logging functions specifically for this purpose. These include:

log: Record a single named value.

log_list: Record a named list of values.

log_row: Record a row with multiple columns.

log_table: Record a dictionary as a table.

log_image: Record an image file or a plot.

In [5]:
import pandas as pd

In [7]:
# create an experiment variable
experiment = Experiment(
    workspace=ws,
    name="my-experiment"
)

# start the experiment
run = experiment.start_logging()

# load the dataset and count number of rows
data = pd.read_csv("dummy_data.csv", header=None)
row_counts = len(data)

# log the row count
run.log('observation', row_counts)

# complete the experiment
run.complete()

## Retreiving and Viewing Logged Metrics

You can view the metrics logged by an experiment run in Azure Machine Learning studio or by using the RunDetails widget in a notebook, as shown here:

# Option 1 - widget

In [8]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Option 2

In [9]:
import json

# get looged metrics
metrics = run.get_metrics()
print(json.dumps(metrics, indent=2))

{
  "observation": 2
}


## Experiment Output Files

In addition to logging metrics, an experiment can generate output files. Often these are trained machine learning models, but you can save any sort of file and make it available as an output of your experiment run. The output files of an experiment are saved in its outputs folder.

The technique you use to add files to the outputs of an experiment depend on how you're running the experiment. The examples shown so far control the experiment lifecycle inline in your code, and when taking this approach you can upload local files to the run's outputs folder by using the Run object's upload_file method in your experiment code as shown here:

In [10]:
run.upload_file(name="outputs/samples.csv", path_or_stream="./dummy_data.csv")

When running an experiment in a remote compute context (which we'll discuss later in this course), any files written to the outputs folder in the compute context are automatically uploaded to the run's outputs folder when the run completes.

## retreiving output files

In [11]:
import json

files = run.get_file_names()
print(json.dumps(files, indent=2))

[
  "outputs/samples.csv"
]


## Running a Script as an Experiment

You can run an experiment inline using the start_logging method of the Experiment object, but it's more common to encapsulate the experiment logic in a script and run the script as an experiment. The script can be run in any valid compute context, making this a more flexible solution for running experiments as scale.



In [12]:
from azureml.core import Experiment, ScriptRunConfig

# create a script config
script_config = ScriptRunConfig(source_directory=".",
script='4_experiment.py')

# submitt the experiment
experiment = Experiment(
    workspace=ws,
    name="my-experiment"
)

run = experiment.submit(config=script_config)
run.wait_for_completion(show_output=True)

RunId: my-experiment_1662108310_5a6273d5
Web View: https://ml.azure.com/runs/my-experiment_1662108310_5a6273d5?wsid=/subscriptions/3ed3266f-ff5e-4b56-b844-7568f3957f98/resourcegroups/am-rg/workspaces/aml-workspace&tid=fd50ea2b-9154-4926-9399-6cc1b0859c88

Streaming azureml-logs/60_control_log.txt

[2022-09-02T08:45:13.896242] Using urllib.request Python 3.0 or later
Streaming log file azureml-logs/60_control_log.txt
Running: ['/bin/bash', '/private/var/folders/3g/fqf9w8vj3kn455_6g9l325kh0000gp/T/azureml_runs/my-experiment_1662108310_5a6273d5/azureml-environment-setup/conda_env_checker.sh']
Starting the daemon thread to refresh tokens in background for process with pid = 65862
Materialized conda environment not found on target: /Users/abhinavmazumdar/.azureml/envs/azureml_f3f7e6c5fb83d94df23933000bf02da3


[2022-09-02T08:45:14.207969] Logging experiment preparation status in history service.
Running: ['/bin/bash', '/private/var/folders/3g/fqf9w8vj3kn455_6g9l325kh0000gp/T/azureml_runs/my

ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}